In [48]:
import json
import argparse
import pandas
import numpy as np
from correctionlib import convert
import correctionlib.schemav2 as cs

In [49]:
ps_csvData = pandas.read_csv("src/BTVNanoCommissioning/data/Prescales/Prescales_PFJet140.csv")
ps_csvData = ps_csvData[ps_csvData['cmsls']!='None']


In [ ]:
### convert ps csv into correctionlib json following along from github.com/cms-nanoAOD/correctionlib/blob/master/data/conversion.py
def get_ps(ps):
    if len(ps) != 1:
        print("Length of ps after selection ", len(ps)) 
        raise ValueError(ps)
    print("Final prescale weight: ", ps.iloc[0]["totprescval"])
    return float(ps.iloc[0]["totprescval"])

In [52]:
def build_lumibins(ps):
    ##### to sort as bin edges properly, starting lumi sections need to be stored as floats 
    print("Path: ", ps["hltpath/prescval"].iloc[0], type(ps["hltpath/prescval"].iloc[0]))
    edges = sorted(set(ps["cmsls"].astype(float)))
    if len(edges)==1:
        return get_ps(ps)
    elif len(edges) > 1:
        #print("Prescale changed")
        edges.append(float('inf'))
        print("Lumi bin edges: ", list(zip(edges[:-1],edges[1:])))
        content = [get_ps(ps[(ps["cmsls"].astype(float)>=lo) & (ps["cmsls"].astype(float)<hi)]) for lo,hi in zip(edges[:-1],edges[1:])]
        print("Prescales: ", content)
        return cs.Binning.parse_obj({
            "nodetype": "binning",
            "input": "lumi",
            "edges": edges,
            "content":content,
            "flow":"clamp"
            })

In [53]:
def build_paths(ps, HLT_paths):
    print("Run: ", ps["# run"].iloc[0], type(ps["# run"].iloc[0]))
    #paths are unique bc of hltpath/lumi --> make array of path name separate
    paths = HLT_paths
    print("Type of path key: ", type(paths[0])) 
    return cs.Category.parse_obj({
        "nodetype": "category",
        "input":"path",
        "content":[{"key":str(path), "value":build_lumibins(ps[ps['hltpath/prescval'].str.contains(path+"_v")])} for path in paths],
    })

In [54]:
HLT_paths =['PFJet140']

In [55]:
def build_runs(ps):
    runs = sorted(ps["# run"].unique())
    print("Selected ", len(runs), ": ", runs)
    return cs.Category.parse_obj({
        "nodetype":"category",
        "input":"run",
        "content":[{"key":int(run), "value":build_paths(ps[ps['# run']==run], HLT_paths)} for run in runs],
    })

In [57]:
psCorr = cs.Correction.parse_obj(
    {
        "version": 2,
        "name": "prescaleWeight",
        "inputs": [
            #cs.Variable(name="pt",   type="real",   description="Leading jet pt"), # try to figure out how to assign path inside correctionlib later 
            {"name":"run", "type":"int"},
            {"name":"path", "type":"string"},
            {"name":"lumi", "type":"real"},
        ],
        "output":{"name":"weight", "type":"real"},
        "data":build_runs(ps_csvData),
    })


Selected  4690 :  [190456, 190459, 190462, 190465, 190482, 190488, 190490, 190491, 190492, 190538, 190539, 190591, 190592, 190593, 190595, 190645, 190646, 190659, 190661, 190662, 190663, 190678, 190679, 190688, 190702, 190703, 190704, 190705, 190706, 190707, 190708, 190710, 190733, 190734, 190735, 190736, 190738, 190782, 190895, 190906, 190945, 190949, 191043, 191046, 191056, 191057, 191062, 191086, 191090, 191201, 191202, 191226, 191242, 191244, 191247, 191248, 191264, 191271, 191276, 191277, 191367, 191387, 191404, 191406, 191411, 191419, 191421, 191424, 191691, 191692, 191693, 191694, 191695, 191697, 191700, 191701, 191718, 191720, 191721, 191723, 191726, 191800, 191808, 191810, 191811, 191830, 191832, 191833, 191834, 191837, 191839, 191842, 191845, 191849, 191856, 191857, 191858, 191859, 193093, 193112, 193115, 193116, 193123, 193124, 193192, 193193, 193207, 193334, 193336, 193541, 193556, 193557, 193575, 193621, 193833, 193834, 193835, 193836, 193878, 193898, 193904, 193905, 19391

Run:  195634 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v5/2 <class 'str'>
Final prescale weight:  400
Run:  195644 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v5/2 <class 'str'>
Final prescale weight:  400
Run:  195645 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v5/2 <class 'str'>
Final prescale weight:  400
Run:  195647 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v5/2 <class 'str'>
Final prescale weight:  400
Run:  195649 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v5/2 <class 'str'>
Final prescale weight:  400
Run:  195655 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v5/2 <class 'str'>
Final prescale weight:  400
Run:  195656 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v5/2 <class 'str'>
Final prescale weight:  400
Run:  195658 <class 'numpy.int64'>
Type of path key:  <

Run:  206866 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v9/2 <class 'str'>
Lumi bin edges:  [(1.0, 216.0), (216.0, inf)]
Final prescale weight:  400
Final prescale weight:  400
Prescales:  [400.0, 400.0]
Run:  206867 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v9/2 <class 'str'>
Final prescale weight:  400
Run:  206868 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v9/2 <class 'str'>
Final prescale weight:  400
Run:  206869 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v9/2 <class 'str'>
Final prescale weight:  400
Run:  206897 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v9/2 <class 'str'>
Lumi bin edges:  [(1.0, 29.0), (29.0, inf)]
Final prescale weight:  600
Final prescale weight:  400
Prescales:  [600.0, 400.0]
Run:  206901 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v9/2 <class 'str'>
Final prescale wei

Run:  251561 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v2/1 <class 'str'>
Final prescale weight:  2000
Run:  251562 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v2/1 <class 'str'>
Final prescale weight:  2000
Run:  251604 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v2/0 <class 'str'>
Lumi bin edges:  [(1.0, 16.0), (16.0, inf)]
Final prescale weight:  0
Final prescale weight:  2000
Prescales:  [0.0, 2000.0]
Run:  251612 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v2/0 <class 'str'>
Lumi bin edges:  [(1.0, 11.0), (11.0, inf)]
Final prescale weight:  0
Final prescale weight:  2000
Prescales:  [0.0, 2000.0]
Run:  251633 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v2/1 <class 'str'>
Final prescale weight:  65537
Run:  251635 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v2/1 <class 'str'>
Final prescale weigh

Run:  257399 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v3/3 <class 'str'>
Final prescale weight:  2400
Run:  257400 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v3/3 <class 'str'>
Final prescale weight:  2400
Run:  257460 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v3/4 <class 'str'>
Final prescale weight:  12000
Run:  257461 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v3/4 <class 'str'>
Lumi bin edges:  [(1.0, 35.0), (35.0, 38.0), (38.0, 69.0), (69.0, inf)]
Final prescale weight:  12000
Final prescale weight:  2400
Final prescale weight:  3000
Final prescale weight:  2400
Prescales:  [12000.0, 2400.0, 3000.0, 2400.0]
Run:  257487 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v3/4 <class 'str'>
Lumi bin edges:  [(1.0, 13.0), (13.0, inf)]
Final prescale weight:  12000
Final prescale weight:  2400
Prescales:  [12000.0, 2400.0]
Run:  257

...

Path:  HLT_PFJet140_v19/0 <class 'str'>
Lumi bin edges:  [(1.0, 3.0), (3.0, inf)]
Final prescale weight:  0
Final prescale weight:  1200
Prescales:  [0.0, 1200.0]
Run:  325159 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v19/0 <class 'str'>
Lumi bin edges:  [(1.0, 2.0), (2.0, 70.0), (70.0, 236.0), (236.0, inf)]
Final prescale weight:  0
Final prescale weight:  1500
Final prescale weight:  1050
Final prescale weight:  900
Prescales:  [0.0, 1500.0, 1050.0, 900.0]
Run:  325167 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v19/1 <class 'str'>
Final prescale weight:  900
Run:  325168 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v19/1 <class 'str'>
Final prescale weight:  900
Run:  325169 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v19/1 <class 'str'>
Final prescale weight:  900
Run:  325170 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v19/1 <c

Run:  356524 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v20/1 <class 'str'>
Final prescale weight:  1500
Run:  356525 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v20/1 <class 'str'>
Lumi bin edges:  [(1.0, 5.0), (5.0, 11.0), (11.0, inf)]
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  1500
Prescales:  [1500.0, 1500.0, 1500.0]
Run:  356526 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v20/1 <class 'str'>
Final prescale weight:  1500
Run:  356527 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v20/1 <class 'str'>
Final prescale weight:  1500
Run:  356528 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v20/1 <class 'str'>
Final prescale weight:  1500
Run:  356529 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v20/1 <class 'str'>
Final prescale weight:  1500
Run:  356530 <class 'numpy.

Run:  357891 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v20/1 <class 'str'>
Final prescale weight:  1500
Run:  357892 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v20/1 <class 'str'>
Lumi bin edges:  [(1.0, 24.0), (24.0, inf)]
Final prescale weight:  1500
Final prescale weight:  1500
Prescales:  [1500.0, 1500.0]
Run:  357893 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v20/1 <class 'str'>
Lumi bin edges:  [(1.0, 3.0), (3.0, 8.0), (8.0, 9.0), (9.0, 11.0), (11.0, inf)]
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  1500
Prescales:  [1500.0, 1500.0, 1500.0, 1500.0, 1500.0]
Run:  357894 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v20/1 <class 'str'>
Final prescale weight:  1500
Run:  357895 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v20/1 <class 

Run:  360090 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v21/1 <class 'str'>
Lumi bin edges:  [(1.0, 110.0), (110.0, 182.0), (182.0, 371.0), (371.0, 687.0), (687.0, inf)]
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  3000
Final prescale weight:  3000
Final prescale weight:  3000
Prescales:  [1500.0, 1500.0, 3000.0, 3000.0, 3000.0]
Run:  360116 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v21/1 <class 'str'>
Lumi bin edges:  [(1.0, 52.0), (52.0, 115.0), (115.0, 121.0), (121.0, 130.0), (130.0, 137.0), (137.0, 157.0), (157.0, 165.0), (165.0, 218.0), (218.0, 237.0), (237.0, 304.0), (304.0, 341.0), (341.0, 420.0), (420.0, inf)]
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  1500
Final presc

Run:  361020 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v21/1 <class 'str'>
Lumi bin edges:  [(1.0, 50.0), (50.0, 62.0), (62.0, 72.0), (72.0, 81.0), (81.0, 89.0), (89.0, 95.0), (95.0, 103.0), (103.0, 109.0), (109.0, inf)]
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  3000
Final prescale weight:  3000
Final prescale weight:  3000
Final prescale weight:  3000
Final prescale weight:  3000
Final prescale weight:  3000
Final prescale weight:  3000
Prescales:  [1500.0, 1500.0, 3000.0, 3000.0, 3000.0, 3000.0, 3000.0, 3000.0, 3000.0]
Run:  361032 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v21/0 <class 'str'>
Final prescale weight:  0
Run:  361044 <class 'numpy.int64'>
Type of path key:  <class 'str'>
Path:  HLT_PFJet140_v21/1 <class 'str'>
Lumi bin edges:  [(1.0, 69.0), (69.0, 77.0), (77.0, inf)]
Final prescale weight:  1500
Final prescale weight:  1500
Final prescale weight:  1500
Prescales:  [

In [61]:
cset = cs.CorrectionSet(
    schema_version = 2,
    corrections = [psCorr],
    description = "prescales for HLT_PFJet140" 
    )

In [62]:
cset

CorrectionSet(schema_version=2, description='prescales for HLT_PFJet140', corrections=[Correction(name='prescaleWeight', description=None, version=2, inputs=[Variable(name='run', type='int', description=None), Variable(name='path', type='string', description=None), Variable(name='lumi', type='real', description=None)], output=Variable(name='weight', type='real', description=None), generic_formulas=None, data=Category(nodetype='category', input='run', content=[CategoryItem(key=190456, value=Category(nodetype='category', input='path', content=[CategoryItem(key='PFJet140', value=Binning(nodetype='binning', input='lumi', edges=[1.0, 9.0, inf], content=[300.0, 10.0], flow='clamp'))], default=None)), CategoryItem(key=190459, value=Category(nodetype='category', input='path', content=[CategoryItem(key='PFJet140', value=Binning(nodetype='binning', input='lumi', edges=[1.0, 5.0, inf], content=[300.0, 10.0], flow='clamp'))], default=None)), CategoryItem(key=190462, value=Category(nodetype='catego

In [63]:
with open("ps_weight_JSON_PFJet140.json", "w") as f:
    f.write(cset.json(exclude_unset=True))

In [71]:
from correctionlib import _core 

In [72]:
evaluator = _core.CorrectionSet.from_file('ps_weight_JSON_PFJet140.json')

In [126]:
evaluator["prescaleWeight"].evaluate(190456,"PFJet140",9.)

10.0

In [121]:
triggers = [
"PFJet140",
]
a = _core.CorrectionSet.from_file(f"src/BTVNanoCommissioning/data/Prescales/ps_weight_JSON_"+str(triggers[0])+".json")

In [123]:
a